
### JAX

https://github.com/jax-ml/jax

JAX es una biblioteca de Python para el cálculo de matrices orientado a aceleradores y la transformación de programas, diseñada para el cálculo numérico de alto rendimiento y el aprendizaje automático a gran escala.

JAX puede diferenciar automáticamente funciones nativas de Python y NumPy. Puede diferenciar mediante bucles, ramas, recursión y cierres, y puede obtener derivadas de derivadas de derivadas. Admite la diferenciación en modo inverso (también conocida como retropropagación) jax.grady la diferenciación en modo directo, y ambas pueden componerse arbitrariamente en cualquier orden.

JAX usa XLA para compilar y escalar tus programas NumPy en TPU, GPU y otros aceleradores de hardware. Puedes compilar tus propias funciones puras con jax.jit. La compilación y la diferenciación automática se pueden componer arbitrariamente.

Profundice un poco más y verá que JAX es realmente un sistema extensible para transformaciones de funciones componibles a escala .


### Documentación 

https://docs.jax.dev/en/latest/

### Guía del desarrollador

https://docs.jax.dev/en/latest/developer.html

### Instalación

In [ ]:
%pip install -U jax

In [ ]:
import jax
import jax.numpy as jnp

# Función de predicción 
def predict(params, inputs):
  for W, b in params:
    outputs = jnp.dot(inputs, W) + b
    inputs = jnp.tanh(outputs)  # inputs to the next layer
  return outputs                # no activation on last layer

# Función de pérdida
def loss(params, inputs, targets):
  preds = predict(params, inputs)
  return jnp.sum((preds - targets)**2)

# grad_loss calcula el gradiente de la función de pérdida con respecto a los parámetros del modelo.
# compilado con jax.jit para optimizar su rendimiento.
# Esto permite que la función se ejecute de manera más eficiente, especialmente en cálculos repetitivos.
# La función jax.grad se utiliza para obtener el gradiente de la función de pérdida.
# Esto es esencial para algoritmos de optimización que ajustan los parámetros del modelo para minimizar
# la pérdida.
grad_loss = jax.jit(jax.grad(loss))  

# perex_grads calcula los gradientes de la función de pérdida para cada ejemplo en un lote de datos.
# Utiliza jax.vmap para vectorizar la función grad_loss, permitiendo así el cálculo eficiente de los 
# gradientes
# cálculo rápido y paralelo de los gradientes para múltiples ejemplos de entrada y sus correspondientes
# objetivos.
perex_grads = jax.jit(jax.vmap(grad_loss, in_axes=(None, 0, 0)))  

### Transformaciones

En esencia, JAX es un sistema extensible para transformar funciones numéricas. Aquí hay tres: jax.grad, jax.jit, y jax.vmap.

**Diferenciación automática con grad**

Uso de jax.grad para calcular eficientemente gradientes de modo inverso:

In [4]:
import jax
import jax.numpy as jnp

def tanh(x):
  y = jnp.exp(-2.0 * x)
  return (1.0 - y) / (1.0 + y)

grad_tanh = jax.grad(tanh)
print(grad_tanh(1.0))
# prints 0.4199743

0.4199743


Diferenciamos cualquier pedido con grad:

In [5]:
print(jax.grad(jax.grad(jax.grad(tanh)))(1.0))
# prints 0.62162673

0.6216267


Podemos usar la diferenciación con el flujo de control de Python:

In [6]:
def abs_val(x):
  if x > 0:
    return x
  else:
    return -x

abs_val_grad = jax.grad(abs_val)
print(abs_val_grad(1.0))   # prints 1.0
print(abs_val_grad(-1.0))  # prints -1.0 (abs_val is re-evaluated)

1.0
-1.0


**Recopilación con jit**

Utilice XLA para compilar sus funciones de extremo a extremo con jit, utilizado como @jitdecorador o como función de orden superior.

In [7]:
import jax
import jax.numpy as jnp

def slow_f(x):
  # Element-wise ops see a large benefit from fusion
  return x * x + x * 2.0

x = jnp.ones((5000, 5000))
fast_f = jax.jit(slow_f)
%timeit -n10 -r3 fast_f(x)
%timeit -n10 -r3 slow_f(x)

The slowest run took 312.96 times longer than the fastest. This could mean that an intermediate result is being cached.
3.54 ms ± 4.95 ms per loop (mean ± std. dev. of 3 runs, 10 loops each)
232 ms ± 21.7 ms per loop (mean ± std. dev. of 3 runs, 10 loops each)


El uso jax.jitrestringe el tipo de flujo de control de Python que la función puede usar; consulte el tutorial sobre Flujo de control y operadores lógicos con JIT para obtener más información.

**Autovectorización convmap**

vmapMapea una función a lo largo de los ejes de una matriz. Pero en lugar de simplemente recorrer las aplicaciones de la función, lo desplaza hacia las operaciones primitivas de la función; por ejemplo, convierte las multiplicaciones matriz-vector en multiplicaciones matriz-matriz para un mejor rendimiento.

El uso vmappuede ahorrarle el tener que llevar dimensiones de lote en su código:

In [8]:
import jax
import jax.numpy as jnp

def l1_distance(x, y):
  assert x.ndim == y.ndim == 1  # only works on 1D inputs
  return jnp.sum(jnp.abs(x - y))

def pairwise_distances(dist1D, xs):
  return jax.vmap(jax.vmap(dist1D, (0, None)), (None, 0))(xs, xs)

xs = jax.random.normal(jax.random.key(0), (100, 3))
dists = pairwise_distances(l1_distance, xs)
dists.shape  # (100, 100)

(100, 100)

Al componer jax.vmapcon jax.grady jax.jit, podemos obtener matrices jacobianas eficientes, o gradientes por ejemplo:

In [9]:
per_example_grads = jax.jit(jax.vmap(jax.grad(loss), in_axes=(None, 0, 0)))

**Escalada**

Consultar

https://github.com/jax-ml/jax?tab=readme-ov-file#scaling
